Funciones para ETL 

In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# Imputación de nulos usando métodos avanzados estadísticos
# -----------------------------------------------------------------------
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer
from sklearn.preprocessing import LabelEncoder

# AB testing
# -----------------------------------------------------------------------
from scipy.stats import ttest_ind
from scipy.stats import levene

# Librerías de visualización
# -----------------------------------------------------------------------
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
#Abrir CSV
df_rh = pd.read_csv("data/recursos-humanos.csv", index_col=0)

In [3]:
#Eliminar columnas

def eliminar_columnas(df_col_sucio):
    columnas_nulos=['Date_of_Birth', 'Age', 'date', 'MarriedID','FromDiversityJobFairID','MaritalStatusID', 'Remuneration','Sex', 'PerfScoreID', 'PositionID', 'Department']
    df_col_sucio.drop(columns=columnas_nulos, inplace=True)
    return df_col_sucio

#Eliminar filas nulas 

def eliminando_10_filas_nulas(df_sucio):
    filas_nulas = df_sucio[(df_sucio["EmpID"].isnull())].index
    df_sucio.drop(filas_nulas, axis = 0, inplace=True)
    return df_sucio

#Cambiar dato a datetime 

def convertir_a_datetime(df, columnas):
    for columna in columnas:
        df[columna] = pd.to_datetime(df[columna], errors='coerce')
    return df

# Unificar nombre del empleado
def mayusculas_todas_palabras(df,columna):
    df[columna] = df[columna].str.title()
    
def nulos_a_unknown(df,columna):
    df[columna] = df[columna].fillna("Unknown")

#Mapeo Department

def categorizar_department_id(df):
    """
    Esta función las siguientes columnas del dataframe() asigna valores
    transformados 
    Parámetros:
    Recibe un dataframe
    Returns:
    Devuelve el dataframe modificado
    """
    mapping = {
        1: 'Admin Offices',
        2: 'Executive Officer',
        3: 'IT/IS',
        4: 'Software Engineering',
        5: 'Production',
        6: 'Sales'
    }
    df['DeptID'] = df['DeptID'].map(mapping)
    return df

#DaysLateLast30, Zip

def nulos_999(df, columns, value):
    for column in columns:
        df[column].fillna(value, inplace=True)
        
#SpecialProjectsCount

def mediana_nulos(df, column):
    mediana_value = df[column].median()
    df[column].fillna(mediana_value, inplace=True)

#ManagerName

def arreglando_ManagerId(df):
    #Hemos comprobado que los nulos de ManagerID corresponden al empleado Webster Butler con ID 39.0
    df["ManagerID"] = df["ManagerID"].fillna(39.0)
    #Cambiamos la columna de ManagerID para tener los nombres de los Managers
    managers = {1.0:'Brandon R. LeBlanc', 
                2.0:'Janet King', 
                3.0:'Brandon R. LeBlanc', 
                4.0:'Simon Roup', 
                5.0:'Jennifer Zamora',
                6.0:'Eric Dougall',
                7.0:'Peter Monroe',
                9.0:'Board of Directors',
                10.0:'Alex Sweetwater',
                11.0:'Amy Dunn',
                12.0:'Brannon Miller',
                13.0:'Brian Champaigne',
                14.0:'David Stanley',
                15.0:'Debra Houlihan',
                16.0:'Elijiah Gray',
                17.0:'John Smith',
                18.0:'Kelley Spirea',
                19.0:'Ketsia Liebig',
                20.0:'Kissy Sullivan',
                21.0:'Lynn Daneault',
                22.0:'Michael Albert',
                30.0:'Michael Albert',
                39.0:'Webster Butler'
                }
    #Sobreescribimos la columna ManagerName para eliminar los valores nulos usando la columna ManagerID
    df["ManagerName"] = df["ManagerID"].map(managers)
    return df

#RaceDesc
def unificar_hispaniclatino(df):
    # Unificar valores 'Yes' y 'yes' como 'Yes' y 'No' y 'no' como 'No' en la columna 'HispanicLatino'
    df['HispanicLatino'] = df['HispanicLatino'].str.lower().replace({'yes': 'Yes', 'no': 'No'})

def imputar_race_desc(df):
    # Si en la columna RaceDesc hay un nulo y en la columna HispanicLatino hay un Yes, imputa un "Hispanic"
    df['RaceDesc'] = df.apply(lambda row: 'Hispanic' if row['HispanicLatino'] == 'Yes' and pd.isnull(row['RaceDesc']) else row['RaceDesc'], axis=1)

def imputar_race_desc_unknown(df):
    # A los nulos restantes, les meto una etiqueta de "Unknown"
    df['RaceDesc'].fillna('Unknown', inplace=True)
    
#Gestionar nulos fechas

def nulos_fecha(df, columnas, fecha_reemplazo):
    for columna in columnas:
        df[columna] = df[columna].fillna(pd.to_datetime(fecha_reemplazo))
    return df

#TermReason, CitizenDesc, RecruitmentSource
def moda_nulos(df, column):
    for columna in column: 
        moda_value = df[columna].mode().iloc[0]
        df[columna].fillna(moda_value, inplace=True)
        
#'DaysLateLast30','SpecialProjectsCount', 'Zip', 'EmpID', 'EmpStatusID', 'Absences'
def convertir_a_entero(df, columnas):
    for columna in columnas:
        df[columna] = df[columna].astype('Int64', errors='ignore')

#EmpSatisfaction

def mapear_satisfaccion_en_columna(df, columna):
    def mapear_satisfaccion(valor):
        if valor == 1.0:
            return 'Very Dissatisfied'
        elif valor == 2.0:
            return 'Dissatisfied'
        elif valor == 3.0:
            return 'Neutral'
        elif valor == 4.0:
            return 'Satisfied'
        elif valor == 5.0:
            return 'Very Satisfied'
        else:
            return 'No answer'

    df[columna] = df[columna].map(mapear_satisfaccion)
    
#Gestión GenderID

def mapear_genero_columna(df, columna):
    def mapear_genero(valor):
        if valor == 1.0:
            return 'Masculine'
        else: 
            return 'Femenine'

    df[columna] = df[columna].map(mapear_genero)

#Termd
    
def mapear_termd_columna(df, columna):
    def mapear_termd(valor):
        if valor == 1.0:
            return 'Yes'
        else: 
            return 'No'

    df[columna] = df[columna].map(mapear_termd)
    
#Salary

def convertir_salary(df, columna):
    df[columna]=df[columna].str.replace(',', '')
    df[columna] = df[columna].astype(float)
    
def columnas_a_minusculas(df):
    """
    Cambia los nombres de las columnas a minusculas
    parametros:
    dataframe del que sacamos las columnas
    diccionario_columnas:Diccionario que mapea nombres de columnas a sus nuevos nombres
    return:
    dataframe modificado
    """
    df.columns = map(str.lower, df.columns)
   
def renombrar_columna(df, nombre_anterior, nombre_nuevo):
    """
    Renombra una columna en el DataFrame
    Parametros:
    - df: DataFrame
    - nombre_anterior: Nombre actual de la columna que va a ser renombrada.
    - nombre_nuevo: Nombre nuevo de la columna.

    Returns:
    - DataFrame actualizado.
    """
    if nombre_anterior in df.columns:   
        df.rename(columns={nombre_anterior: nombre_nuevo}, inplace=True)
        return df
    else:
        return df
    
    


In [4]:
df_rh=eliminar_columnas(df_rh)

#Eliminar filas nulas 

df_rh=eliminando_10_filas_nulas(df_rh)

#Cambiar dato a datetime 

df_rh=convertir_a_datetime(df_rh, ['DateofTermination','DateofHire','LastPerformanceReview_Date', 'DOB'])
  

# Unificar nombre del empleado
mayusculas_todas_palabras(df_rh,'Employee_Name')

    
nulos_a_unknown(df_rh,'Employee_Name')
    

#Mapeo Department

df_rh=categorizar_department_id(df_rh)
   

#DaysLateLast30, Zip

nulos_999(df_rh, ['DaysLateLast30', 'Zip'], 999)
  
        
#SpecialProjectsCount

mediana_nulos(df_rh, 'SpecialProjectsCount')
    

#ManagerName

df_rh=arreglando_ManagerId(df_rh)
    

#RaceDesc
unificar_hispaniclatino(df_rh)
   

imputar_race_desc(df_rh)


imputar_race_desc_unknown(df_rh)

    
#Gestionar nulos fechas

df_rh=nulos_fecha(df_rh, ['DateofHire','LastPerformanceReview_Date'], '1900-01-01')

#TermReason, CitizenDesc, RecruitmentSource
moda_nulos(df_rh, ['TermReason', 'CitizenDesc', 'RecruitmentSource'])

#'DaysLateLast30','SpecialProjectsCount', 'Zip', 'EmpID', 'EmpStatusID', 'Absences'

convertir_a_entero(df_rh, ['DaysLateLast30','SpecialProjectsCount', 'Zip', 'EmpID', 'EmpStatusID', 'Absences'])

#EmpSatisfaction

mapear_satisfaccion_en_columna(df_rh, 'EmpSatisfaction')

#GenderID

mapear_genero_columna(df_rh, 'GenderID')

#Termd

mapear_termd_columna(df_rh, 'Termd')

#Salary 
convertir_salary(df_rh, 'Salary')

#Cambiar nombres de las columnas
columnas_a_minusculas(df_rh)

df_rh = renombrar_columna(df_rh,'dob','date_of_birth')
df_rh = renombrar_columna(df_rh,'department','xdepartment')
df_rh= renombrar_columna(df_rh,'deptid','department')

   

C:\Users\nalon\AppData\Local\Temp\ipykernel_6172\2317635789.py:19: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[columna] = pd.to_datetime(df[columna], errors='coerce')


In [5]:
df_rh.sample(7)

,employee_name,empid,genderid,empstatusid,department,salary,termd,position,state,zip,date_of_birth,maritaldesc,citizendesc,hispaniclatino,racedesc,dateofhire,dateoftermination,termreason,employmentstatus,managername,managerid,recruitmentsource,performancescore,engagementsurvey,empsatisfaction,specialprojectscount,lastperformancereview_date,dayslatelast30,absences
65,"Degweck, James",10070,Masculine,5,Production,55722.0,Yes,Production Technician I,MA,1810,1977-10-31,Married,US Citizen,No,Unknown,2011-05-16,2016-06-08,unhappy,Voluntarily Terminated,Webster Butler,39.0,Indeed,Fully Meets,5.00,Satisfied,0,2016-04-02,999,14
31,"Bugali, Josephine",10203,Femenine,3,Production,64375.0,No,Production Technician I,MA,2043,2069-10-30,Separated,US Citizen,No,Black or African American,2013-11-11,NaT,Terminated,Active,Kissy Sullivan,20.0,Diversity Job Fair,Fully Meets,3.50,Very Satisfied,0,2019-01-21,0,17
19,Unknown,10277,Masculine,3,Production,53250.0,No,Production Technician I,MA,2452,1979-04-06,Single,US Citizen,No,Unknown,2013-07-08,NaT,N/A-StillEmployed,Active,Webster Butler,39.0,LinkedIn,Fully Meets,4.20,Satisfied,0,2019-01-11,0,13
45,"Chigurh, Anton",10200,Masculine,1,Sales,66808.0,No,Area Sales Manager,TX,78207,2070-06-11,Single,Eligible NonCitizen,No,Black or African American,1900-01-01,NaT,N/A-StillEmployed,Active,Lynn Daneault,21.0,Employee Referral,Fully Meets,3.00,Very Satisfied,0,2019-01-19,999,17
2,"Akinkuolie, Sarah",10196,Femenine,5,Production,64955.0,Yes,Production Technician II,MA,1810,1988-09-19,Married,US Citizen,No,Unknown,1900-01-01,2012-09-24,hours,Voluntarily Terminated,Kissy Sullivan,20.0,LinkedIn,Fully Meets,3.02,Neutral,0,2012-05-15,999,3
26,Unknown,10081,Femenine,1,Admin Offices,106367.0,No,Sr. Accountant,MA,2468,1987-04-04,Married,US Citizen,No,Black or African American,2015-02-16,NaT,N/A-StillEmployed,Active,Brandon R. LeBlanc,3.0,Diversity Job Fair,Fully Meets,5.00,Satisfied,0,2019-02-18,0,4
298,Unknown,10172,Masculine,1,IT/IS,84903.0,No,Senior BI Developer,MA,1887,1981-07-08,Single,US Citizen,No,Unknown,2017-02-15,NaT,N/A-StillEmployed,Active,Brian Champaigne,13.0,Indeed,Fully Meets,3.42,Satisfied,0,2019-01-04,999,17


In [6]:
df_rh.isna().sum()

employee_name                   0
empid                           0
genderid                        0
empstatusid                     0
department                      0
salary                          0
termd                           0
position                        0
state                           0
zip                             0
date_of_birth                   0
maritaldesc                     0
citizendesc                     0
hispaniclatino                  0
racedesc                        0
dateofhire                      0
dateoftermination             209
termreason                      0
employmentstatus                0
managername                     0
managerid                       0
recruitmentsource               0
performancescore                0
engagementsurvey                0
empsatisfaction                 0
specialprojectscount            0
lastperformancereview_date      0
dayslatelast30                  0
absences                        0
dtype: int64